In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras.layers import Dense
from keras_tuner import RandomSearch, Hyperband

import pandas as pd 

In [2]:
#  Import and read the charity_data.csv.
application_df = pd.read_csv("Starter_Code/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df= application_df.drop(["EIN", "NAME"], axis = 1)

In [4]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [5]:
# Look at APPLICATION_TYPE value counts for binning
application_counts=application_df["APPLICATION_TYPE"].value_counts()

In [6]:
application_types_to_replace = (list(application_counts[application_counts < 500].index))
application_types_to_replace

['T9', 'T13', 'T12', 'T2', 'T25', 'T14', 'T29', 'T15', 'T17']

In [7]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [8]:
class_count = application_df["CLASSIFICATION"].value_counts()

In [9]:
class_count[class_count > 1]

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64

In [10]:
# Determine which values to replace if counts are less than 1000
classifications_to_replace = list(class_count[class_count < 1000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [11]:
# Convert categorical data to numeric with `pd.get_dummies`
dummified_df = pd.get_dummies(application_df, dtype=int,drop_first= True)

In [12]:
# Split our preprocessed data into our features and target arrays
X = dummified_df.drop("IS_SUCCESSFUL", axis=1)
y = dummified_df["IS_SUCCESSFUL"]

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=1)

In [13]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [14]:
X_train.shape

(25724, 36)

----------
### Attemp 1
----------

In [15]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
model_nn = tf.keras.models.Sequential()

# First hidden layer
model_nn.add(Dense(units = 512, activation = "sigmoid", input_dim = X_train.shape[1]))

# Second hidden layer
model_nn.add(Dense(units = 448, activation = "sigmoid"))
                               

# Output layer
model_nn.add(Dense(units = 1, activation = "sigmoid"))

# Check the structure of the model
model_nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               18944     
                                                                 
 dense_1 (Dense)             (None, 448)               229824    
                                                                 
 dense_2 (Dense)             (None, 1)                 449       
                                                                 
Total params: 249217 (973.50 KB)
Trainable params: 249217 (973.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [16]:
# Compile 
model_nn.compile(loss = "binary_crossentropy", optimizer= keras.optimizers.Adam(learning_rate=0.001), metrics = ["accuracy"])

In [17]:
# Train
model_nn.fit(X_train_scaled, y_train, epochs = 50)

Epoch 1/50
804/804 [==============================] - 3s 3ms/step - loss: 0.6013 - accuracy: 0.6971
Epoch 2/50
804/804 [==============================] - 4s 5ms/step - loss: 0.5775 - accuracy: 0.7187
Epoch 3/50
804/804 [==============================] - 4s 5ms/step - loss: 0.5700 - accuracy: 0.7221
Epoch 4/50
804/804 [==============================] - 4s 5ms/step - loss: 0.5666 - accuracy: 0.7240
Epoch 5/50
804/804 [==============================] - 5s 6ms/step - loss: 0.5647 - accuracy: 0.7245
Epoch 6/50
804/804 [==============================] - 4s 5ms/step - loss: 0.5637 - accuracy: 0.7266
Epoch 7/50
804/804 [==============================] - 4s 5ms/step - loss: 0.5613 - accuracy: 0.7266
Epoch 8/50
804/804 [==============================] - 4s 5ms/step - loss: 0.5594 - accuracy: 0.7274
Epoch 9/50
804/804 [==============================] - 4s 5ms/step - loss: 0.5599 - accuracy: 0.7265
Epoch 10/50
804/804 [==============================] - 4s 5ms/step - loss: 0.5579 - accuracy: 0.7287

In [18]:
# Evaluate
model_loss, model_accuracy = model_nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f" loss : {round(model_loss,2)*100}%, accuray : {round(model_accuracy,2)*100}%")

268/268 - 1s - loss: 0.5564 - accuracy: 0.7304 - 1s/epoch - 5ms/step
 loss : 56.00000000000001%, accuray : 73.0%


--------------
### Attempt 2
---------------

In [19]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
model_nn = tf.keras.models.Sequential()

# First hidden layer
model_nn.add(Dense(units = 512, activation = "sigmoid", input_dim = X_train.shape[1]))

# Second hidden layer
model_nn.add(Dense(units = 256, activation = "sigmoid"))

model_nn.add(Dense(units = 128, activation = "sigmoid"))

model_nn.add(Dense(units = 64, activation = "sigmoid"))

model_nn.add(Dense(units = 32, activation = "sigmoid"))

model_nn.add(Dense(units = 16, activation = "sigmoid"))

model_nn.add(Dense(units = 8, activation = "sigmoid"))

model_nn.add(Dense(units = 4, activation = "sigmoid"))
                       
        
# Output layer
model_nn.add(Dense(units = 1, activation = "sigmoid"))

# Check the structure of the model
model_nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 512)               18944     
                                                                 
 dense_4 (Dense)             (None, 256)               131328    
                                                                 
 dense_5 (Dense)             (None, 128)               32896     
                                                                 
 dense_6 (Dense)             (None, 64)                8256      
                                                                 
 dense_7 (Dense)             (None, 32)                2080      
                                                                 
 dense_8 (Dense)             (None, 16)                528       
                                                                 
 dense_9 (Dense)             (None, 8)                

In [20]:
# Compile 
model_nn.compile(loss = "binary_crossentropy", optimizer= "adam", metrics = ["accuracy"])

# Train
model_nn.fit(X_train_scaled, y_train, epochs = 10)

Epoch 1/10
804/804 [==============================] - 7s 5ms/step - loss: 0.6877 - accuracy: 0.5432
Epoch 2/10
804/804 [==============================] - 4s 5ms/step - loss: 0.6911 - accuracy: 0.5335
Epoch 3/10
804/804 [==============================] - 4s 6ms/step - loss: 0.6908 - accuracy: 0.5349
Epoch 4/10
804/804 [==============================] - 5s 6ms/step - loss: 0.6088 - accuracy: 0.7129
Epoch 5/10
804/804 [==============================] - 5s 6ms/step - loss: 0.5955 - accuracy: 0.7168
Epoch 6/10
804/804 [==============================] - 5s 6ms/step - loss: 0.5947 - accuracy: 0.7172
Epoch 7/10
804/804 [==============================] - 5s 6ms/step - loss: 0.5928 - accuracy: 0.7193
Epoch 8/10
804/804 [==============================] - 4s 5ms/step - loss: 0.5862 - accuracy: 0.7235
Epoch 9/10
804/804 [==============================] - 4s 5ms/step - loss: 0.5822 - accuracy: 0.7240
Epoch 10/10
804/804 [==============================] - 4s 5ms/step - loss: 0.5809 - accuracy: 0.7246

In [21]:
# Evaluate
model_loss, model_accuracy = model_nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f" loss : {round(model_loss,2)*100}%, accuray : {round(model_accuracy,2)*100}%")

268/268 - 1s - loss: 0.5815 - accuracy: 0.7255 - 745ms/epoch - 3ms/step
 loss : 57.99999999999999%, accuray : 73.0%


----------
### Attempt 3
------------

In [22]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
model_nn = tf.keras.models.Sequential()

# First hidden layer
model_nn.add(Dense(units = 100, activation = "relu", input_dim = X_train.shape[1]))

# Second hidden layer
model_nn.add(Dense(units = 80, activation = "sigmoid"))


model_nn.add(Dense(units = 60, activation = "sigmoid"))


model_nn.add(Dense(units = 20, activation = "sigmoid"))

# Output layer
model_nn.add(Dense(units = 1, activation = "sigmoid"))

# Check the structure of the model
model_nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 100)               3700      
                                                                 
 dense_13 (Dense)            (None, 80)                8080      
                                                                 
 dense_14 (Dense)            (None, 60)                4860      
                                                                 
 dense_15 (Dense)            (None, 20)                1220      
                                                                 
 dense_16 (Dense)            (None, 1)                 21        
                                                                 
Total params: 17881 (69.85 KB)
Trainable params: 17881 (69.85 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [23]:
# Compile 
model_nn.compile(loss = "binary_crossentropy", optimizer= keras.optimizers.Adam(learning_rate=0.0003), metrics = ["accuracy"])

# Train
model_nn.fit(X_train_scaled, y_train, epochs = 100)

Epoch 1/100
804/804 [==============================] - 4s 2ms/step - loss: 0.6144 - accuracy: 0.6758
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5699 - accuracy: 0.7289
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5643 - accuracy: 0.7301
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5597 - accuracy: 0.7306
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5563 - accuracy: 0.7310
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5542 - accuracy: 0.7311
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5530 - accuracy: 0.7297
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5521 - accuracy: 0.7323
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5510 - accuracy: 0.7312
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5503 - accura

804/804 [==============================] - 2s 2ms/step - loss: 0.5407 - accuracy: 0.7372
Epoch 82/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5406 - accuracy: 0.7375
Epoch 83/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5404 - accuracy: 0.7384
Epoch 84/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5400 - accuracy: 0.7378
Epoch 85/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5406 - accuracy: 0.7372
Epoch 86/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5403 - accuracy: 0.7371
Epoch 87/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5402 - accuracy: 0.7379
Epoch 88/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5402 - accuracy: 0.7382
Epoch 89/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5401 - accuracy: 0.7379
Epoch 90/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5400 - accuracy: 

In [24]:
# Evaluate
model_loss, model_accuracy = model_nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f" loss : {round(model_loss,1)*100}%, accuray : {round(model_accuracy,2)*100}%")

268/268 - 1s - loss: 0.5522 - accuracy: 0.7291 - 688ms/epoch - 3ms/step
 loss : 60.0%, accuray : 73.0%


In [25]:
# Export our model to HDF5 file
model_nn.save("AlphabetSoupCharity_Optimization.keras")